# Baseline MLP Classification Experiment

In [17]:
import pandas as pd

## Data Handling

In [18]:
# load data
tag_metal = "../../my_data/identification-dataset/nlos/anTag/metal/data1/metal-anTag-data1_data.csv"
df_tag = pd.read_csv(tag_metal)
df_tag.head()

,tdoa12,tdoa21,snr_an1,power_dif_an1,snr_an2,power_dif_an2,an1_rx_snr,an1_rx_powerdif,an1_tof,an2_rx_snr,an2_rx_powerdif,an2_tof
0,-0.319040,0.380033,24.450001,23.648659,153.833328,11.635933,132.000000,14.551033,5.037270,168.541672,13.384590,5.018502
1,-0.319040,0.380033,24.450001,23.648659,162.750000,11.179535,172.666672,14.281364,5.056036,176.125000,13.252098,5.041961
2,-0.319040,0.319040,14.928572,25.917366,162.750000,11.179535,203.149994,14.264801,5.088879,148.850006,13.729012,5.065420
3,-0.464485,0.319040,20.607143,26.443871,187.500000,11.430855,159.000000,13.419594,5.084187,174.600006,13.297188,5.056036
4,-0.300273,0.323732,26.416666,25.613457,161.416672,11.633659,226.899994,13.669937,5.060728,195.600006,13.437340,5.051345


In [19]:
# splitting in anchor information
df_an1 = df_tag[['tdoa12', 'snr_an1', 'power_dif_an1', 'an1_rx_snr', 'an1_rx_powerdif', 'an1_tof']].copy()
df_an2 = df_tag[['tdoa21', 'snr_an2', 'power_dif_an2', 'an2_rx_snr', 'an2_rx_powerdif', 'an2_tof']].copy()

In [20]:
# renaming cols
df_an1.rename({'tdoa12':'tdoa', 
                   'snr_an1':'snr_an', 
                   'power_dif_an1':'power_dif', 
                   'an1_rx_snr':'rx_snr', 
                   'an1_rx_powerdif':'rx_powerdif', 
                   'an1_tof':'tof'}, axis=1, inplace=True)

df_an2.rename({'tdoa21':'tdoa',
                   'snr_an2':'snr_an', 
                   'power_dif_an2':'power_dif', 
                   'an2_rx_snr':'rx_snr', 
                   'an2_rx_powerdif':'rx_powerdif', 
                   'an2_tof':'tof'}, axis=1, inplace=True)

In [21]:
# adding information of nlos causing object
df_an1['NLOS_material'] = 1 # metal
df_an2['NLOS_material'] = 0 # nothing

In [22]:
df = pd.concat([df_an1, df_an1], ignore_index=True, sort=False)
df.head()

,tdoa,snr_an,power_dif,rx_snr,rx_powerdif,tof,NLOS_material
0,-0.319040,24.450001,23.648659,132.000000,14.551033,5.037270,1
1,-0.319040,24.450001,23.648659,172.666672,14.281364,5.056036,1
2,-0.319040,14.928572,25.917366,203.149994,14.264801,5.088879,1
3,-0.464485,20.607143,26.443871,159.000000,13.419594,5.084187,1
4,-0.300273,26.416666,25.613457,226.899994,13.669937,5.060728,1


In [23]:
df.dropna(inplace=True)

## Classifying 

In [24]:
# Import modules from Scikit-learn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split   # Import train_test_split function
from sklearn import metrics   # import metrics modules for accuracy calculation
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [25]:
# Create a pipeline object for the model
pipe_MLP = make_pipeline(StandardScaler(),
                         MLPClassifier(solver='adam',
                                       hidden_layer_sizes=(100,100,100,), # 3 hidden layers with (100x100x100) neurons
                                       random_state=0,
                                       max_iter=500,           # TODO: tune it later
                                       verbose=True
                                       )
                         )

In [26]:
# Extracted features 
X = df[['tdoa', 'snr_an', 'power_dif', 'rx_snr', 'rx_powerdif', 'tof', 'NLOS_material']]
y = df['NLOS_material'] # Labels

In [27]:
# Split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # 70% training and 30% test

In [28]:
# Train the whole pipeline
pipe_MLP.fit(X_train, y_train)

Iteration 1, loss = 0.49575595
Iteration 2, loss = 0.08588530
Iteration 3, loss = 0.00607879
Iteration 4, loss = 0.00107773
Iteration 5, loss = 0.00054503
Iteration 6, loss = 0.00040640
Iteration 7, loss = 0.00033829
Iteration 8, loss = 0.00029273
Iteration 9, loss = 0.00025289
Iteration 10, loss = 0.00021022
Iteration 11, loss = 0.00017196
Iteration 12, loss = 0.00014148
Iteration 13, loss = 0.00011911
Iteration 14, loss = 0.00010358
Iteration 15, loss = 0.00009264
Iteration 16, loss = 0.00008508
Iteration 17, loss = 0.00007965
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=500,
                               random_state=0, verbose=True))])

In [29]:
y_pred = pipe_MLP.predict(X_test)

In [30]:
print(f'Test Accuracy: {metrics.accuracy_score(y_test, y_pred) * 100}%')

Test Accuracy: 100.0%
